In [1]:
import numpy as np
import pandas as pd
from simple.ohlc import renko, midPrice
from simple.chart import chartFigure
from simple.funcs import symlog

In [2]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)
len(T)

907307

In [3]:
Buy = T[T.Size > 0]
Sell = T[T.Size < 0]
Buy[:5]

rec.array([('2022-05-08T00:01:00.335000', 2636.,  6.),
           ('2022-05-08T00:01:00.335000', 2636.,  4.),
           ('2022-05-08T00:01:00.668000', 2636., 96.),
           ('2022-05-08T00:01:00.668000', 2636.,  2.),
           ('2022-05-08T00:01:00.668000', 2636.,  2.)],
          dtype=[('DateTime', '<M8[us]'), ('Price', '<f8'), ('Size', '<f8')])

In [4]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S[:5]

rec.array([('2022-05-08T00:01:00.007000', 2636., 2635.99),
           ('2022-05-08T00:01:00.043000', 2636., 2635.99),
           ('2022-05-08T00:01:00.079000', 2636., 2635.99),
           ('2022-05-08T00:01:00.117000', 2636., 2635.99),
           ('2022-05-08T00:01:00.149000', 2636., 2635.99)],
          dtype={'names': ['DateTime', 'Ask', 'Bid'], 'formats': ['<M8[us]', '<f4', '<f4'], 'offsets': [0, 8, 16], 'itemsize': 24})

In [5]:
len(T), len(S)

(907307, 600964)

In [7]:
# indexed chart
fig = chartFigure(
    Tick=dict(y=T.Price, x=T.DateTime, color='gray', opacity=0.4),
    Sell=dict(y=Sell.Price, x=Sell.DateTime, color='red', mode='markers', size=-symlog(Sell.Size)),
    Buy=dict(y=Buy.Price, x=Buy.DateTime, color='green', mode='markers', size=symlog(Buy.Size))
)
fig

FigureWidgetResampler({
    'data': [{'line': {'color': 'gray'},
              'name': '<b style="color:sandybrown">[R]</b> Tick <i style="color:#fc9944">~22s</i>',
              'opacity': 0.4,
              'type': 'scattergl',
              'uid': '4636d679-bbee-4028-8a3b-7b86928872d8',
              'x': array([datetime.datetime(2022, 5, 8, 0, 1, 0, 335000),
                          datetime.datetime(2022, 5, 8, 0, 1, 5, 755000),
                          datetime.datetime(2022, 5, 8, 0, 1, 31, 509000), ...,
                          datetime.datetime(2022, 5, 8, 5, 58, 53, 508000),
                          datetime.datetime(2022, 5, 8, 5, 59, 13, 945000),
                          datetime.datetime(2022, 5, 8, 5, 59, 59, 909999)], dtype=object),
              'y': array([2636.  , 2635.6 , 2639.29, ..., 2537.56, 2536.07, 2537.16])},
             {'marker': {'color': 'red',
                         'size': array([4.58351894, 5.91265489, 1.69314718, ..., 6.22035583, 7.3403593 ,
   

In [ ]:
C = renko(T, 20)
C[:10]

In [ ]:
fig.layout.shapes = [{
     'fillcolor': 'green' if c[3] > c[2] else 'red',
     'line': {'color': 'black', 'width': 1},
     'opacity': 0.2,
     'type': 'rect',
     'x0': c[0],
     'x1': c[1],
     'y0': c[2],
     'y1': c[3]
 } for c in C]